<a href="https://colab.research.google.com/github/sushant822/big-data-challenge/blob/main/big_data_challenge_software.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-2.4.7'
spark_version = 'spark-2.4.7'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bioni

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-11-01 20:12:41--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.96MB/s    in 0.2s    

2020-11-01 20:12:41 (4.96 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://classdb27oct2020.s3-us-west-1.amazonaws.com/amazon_reviews_us_Digital_Software_v1_00.tsv"
spark.sparkContext.addFile(url)
digital_software_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Software_v1_00.tsv"), 
                                        sep="\t", header=True, inferSchema=True)

# Show DataFrame
digital_software_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   17747349|R2EI7QLPK4LF7U|B00U7LCE6A|     106182406|CCleaner Free [Do...|Digital_Software|          4|            0|          0|   N|                Y|          Four Stars|      So far so good|2015-08-31 00:00:00|
|         US|   10956619|R1W5OMFK1Q3I3O|B00HRJMOM4|     162269768|ResumeMaker Profe...|D

In [5]:
print(f"Original count: {digital_software_df.count()}")

Original count: 102084


In [6]:
# Print schema
digital_software_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [7]:
from pyspark.sql.functions import to_date
# Review DataFrame
review_id_df = digital_software_df.select(["review_id", "customer_id", "product_id", "product_parent", 
                                              to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2EI7QLPK4LF7U|   17747349|B00U7LCE6A|     106182406| 2015-08-31|
|R1W5OMFK1Q3I3O|   10956619|B00HRJMOM4|     162269768| 2015-08-31|
| RPZWSYWRP92GI|   13132245|B00P31G9PQ|     831433899| 2015-08-31|
|R2WQWM04XHD9US|   35717248|B00FGDEPDY|     991059534| 2015-08-31|
|R1WSPK2RA2PDEF|   17710652|B00FZ0FK0U|     574904556| 2015-08-31|
|R11JVGRZRHTDAS|   42392705|B004KPKSRQ|     306022575| 2015-08-31|
|R2B8468OKXXYE2|   52845868|B00B1TFNTW|      54873662| 2015-08-31|
|R2HGGCCZSSNUCB|   15696503|B00M9GTJLY|     103182180| 2015-08-31|
| REEE4LHSVPRV9|    9723928|B00H9A60O4|     608720080| 2015-08-31|
|R25OMUUILFFHI9|   23522877|B008XAXAC4|      87969525| 2015-08-31|
|R2966PB8UBD5BM|   17022093|B00MHZ6Z64|     249773946| 2015-08-31|
|R1OU91L2G5H6H1|   11635690|B00OPCQ70Q|     956532818| 2015-08

In [8]:
products_df = digital_software_df.select(["product_id", "product_title"]).drop_duplicates()

In [9]:
reviews_df = digital_software_df.select(["review_id", "review_headline", "review_body"])
reviews_df.show()

+--------------+--------------------+--------------------+
|     review_id|     review_headline|         review_body|
+--------------+--------------------+--------------------+
|R2EI7QLPK4LF7U|          Four Stars|      So far so good|
|R1W5OMFK1Q3I3O|         Three Stars|Needs a little mo...|
| RPZWSYWRP92GI|            One Star|      Please cancel.|
|R2WQWM04XHD9US|  Works as Expected!|  Works as Expected!|
|R1WSPK2RA2PDEF|Great antivirus. ...|I've had Webroot ...|
|R11JVGRZRHTDAS|Great choice in s...|EXCELLENT softwar...|
|R2B8468OKXXYE2|Munch, munch, mun...|The variations cr...|
|R2HGGCCZSSNUCB|Horrible! Would n...|Horrible!  Would ...|
| REEE4LHSVPRV9|            One Star|     Waste of time .|
|R25OMUUILFFHI9|Very good suite o...|Work as easy as o...|
|R2966PB8UBD5BM|          Five Stars|    Works perfectly!|
|R1OU91L2G5H6H1|            One Star|Will not open on ...|
|R3M6YQVMXWGTR6|Purchased this on...|Purchased this on...|
|R2M8VZGO4BFN9J|            One Star|God awful and did..

In [10]:
customers_df = digital_software_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   46639223|             1|
|   50407297|             1|
|   52707015|             1|
|   21817779|             1|
|     178254|             1|
|    1442229|             1|
|   12811624|             1|
|   40840841|             1|
|   48319420|             1|
|   16205135|             1|
|   12115907|             1|
|   30190304|             1|
|   43013424|             1|
|   51495952|             1|
|   16558777|             1|
|   53013109|             2|
|   50065664|             1|
|    3534576|             1|
|   36456998|             1|
|   31609314|             1|
+-----------+--------------+
only showing top 20 rows



In [11]:
vine_df = digital_software_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2EI7QLPK4LF7U|          4|            0|          0|   N|
|R1W5OMFK1Q3I3O|          3|            0|          0|   N|
| RPZWSYWRP92GI|          1|            1|          2|   N|
|R2WQWM04XHD9US|          5|            0|          0|   N|
|R1WSPK2RA2PDEF|          4|            1|          2|   N|
|R11JVGRZRHTDAS|          5|            4|          4|   N|
|R2B8468OKXXYE2|          1|            1|          1|   N|
|R2HGGCCZSSNUCB|          1|            0|          0|   N|
| REEE4LHSVPRV9|          1|            0|          0|   N|
|R25OMUUILFFHI9|          5|            0|          0|   N|
|R2966PB8UBD5BM|          5|            0|          0|   N|
|R1OU91L2G5H6H1|          1|            0|          2|   N|
|R3M6YQVMXWGTR6|          5|            0|          0|   N|
|R2M8VZGO4BFN9J|          1|            

In [12]:
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.ct26r4oxbudr.us-west-1.rds.amazonaws.com:5432/assignment_big_data"
config = {"user":"root", "password": "12345678", "driver":"org.postgresql.Driver"}

In [13]:
review_id_df.write.jdbc(url=jdbc_url, table='software_review', mode=mode, properties=config)

In [14]:
products_df.write.jdbc(url=jdbc_url, table='software_products', mode=mode, properties=config)

In [15]:
customers_df.write.jdbc(url=jdbc_url, table='software_customers', mode=mode, properties=config)

In [16]:
vine_df.write.jdbc(url=jdbc_url, table='software_vines', mode=mode, properties=config)